<a href="https://colab.research.google.com/github/AshuKV/Reddit-Flair-Predictor/blob/master/Project_Reddit_Flair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Reddit Flair Detector

A Reddit Flair Detector using Machine Learning Algorithms

### Install Required Modules

Install Wrapper for Reddit's API and Drive's API. 
Further install Library for Unsupervised Modeling and NLP.


In [2]:
!pip install praw pandas numpy scikit-learn matplotlib scipy PyDrive
!pip install gensim nltk

     |████████████████████████████████| 153kB 7.5MB/s 
     |████████████████████████████████| 204kB 14.9MB/s 


### Import Modules

Import all the required libraries using the import command.

In [3]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import praw
import pandas as pd
import datetime as dt
import logging
import numpy as np
from numpy import random
import gensim
import nltk
nltk.download('all')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

### Variable Declarations

Enter all the required flairs that has to be detected using in a list.

In [0]:
flairs = ["AskIndia", "Non-Political", "[R]eddiquette", "Scheduled", "Photography", "Science/Technology", "Politics", "Business/Finance", "Policy/Economy", "Sports", "Food", "AMA"]
#Using Regular Expression to compile all the variables and withdraw any bad words.
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
#Storing all the stopwords in a list named STOPWORDS
STOPWORDS = set(stopwords.words('english'))

### Util Functions

In [0]:
#Function to return the local date 
def get_date(created):
    return dt.datetime.fromtimestamp(created)

#Function to convert a variable to string type
def string_form(value):
    return str(value)

#Function for parsing and cleaning the text
def clean_text(text): 
    text = BeautifulSoup(text, "lxml").text
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

An attempt was made for getting comments from the reddit flair for which the StreamHandler class would return a logger with prawcore, which ultimately failed.

In [0]:
'''import logging
handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
logger = logging.getLogger('prawcore')
logger.setLevel(logging.DEBUG)
logger.addHandler(handler)'''

Getting the current working directory.

In [6]:
pwd

'/content'

### Collecting Reddit India Data  from r/india.

Using PRAW module to get 50 India Reddit data for each of 12 flairs as selected above 

In [0]:
#Enter the 14-digit script, 27-digits secret key, user agent, username and password, after logging on to Reddit
reddit = praw.Reddit(client_id='F7Tj27YBPXb1Bw', client_secret='d9HY3XFSHxOssTmZ1uZPfj6op1c', user_agent='ashuv', username='Ashuv12', password='Priy@m@123')
#getting to Subreddit "india"
subreddit = reddit.subreddit('india')
#Initialze the dictionary for storing all the attributes of the flairs.
topics_dict = {"flair":[], "title":[], "score":[], "id":[], "url":[], "comms_num": [], "created": [], "body":[], "author":[], "comments":[]}
#Iterating over the list "flairs"
for flair in flairs:
  get_subreddits = subreddit.search(flair, limit=50)
  #Iterating over the 10 subreddits flairs
  for submission in get_subreddits:
    #Storing the values for particular keys
    topics_dict["flair"].append(flair)
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)
    topics_dict["author"].append(submission.author)
    #getting only the top level comments, as the comments for a particalr post could be very large. 
    submission.comments.replace_more(limit=None)
    comment = ''
    for top_level_comment in submission.comments:
      comment = comment + ' ' + top_level_comment.body
    topics_dict["comments"].append(comment)


Checking if all the keys have same no. of values, since the dictionary has to converted to DataFrame, and it requires having same length of column soze for each attribute.

In [12]:
print(len(topics_dict["flair"]))

568


In [13]:
print(len(topics_dict["title"]))

568


In [14]:
print(len(topics_dict["score"]))

568


In [15]:
print(len(topics_dict["id"]))

568


In [16]:
print(len(topics_dict["url"]))

568


In [17]:
print(len(topics_dict["comms_num"]))

568


In [18]:
print(len(topics_dict["created"]))

568


In [19]:
print(len(topics_dict["body"]))

568


In [20]:
print(len(topics_dict["author"]))

568


In [21]:
print(len(topics_dict["comments"]))

568


In [22]:
for x in topics_dict:
  print(x)

flair
title
score
id
url
comms_num
created
body
author
comments


Printing the dictionary's keys and its corresponding values.

In [23]:
for x, y in topics_dict.items():
  print(x, y)

flair ['AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'AskIndia', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'Non-Political', 'N

Creating a 2-d tabular data using Pandas.

In [0]:
topics_data = pd.DataFrame(topics_dict)

Creating a reddit-india-data CSV file by its time-created.

In [0]:
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)
del topics_data['created']
topics_data.to_csv('reddit-india-data.csv', index=False)

### Loading r/india Data
Reading CSV files and get top 25 rows.

In [0]:
topics_data = pd.read_csv('reddit-india-data.csv')
topics_data.head(25)

### Text Pre-Processing
Analysing the data for Exploratory Data Analysis.
The titles, cooments and body have to cleaned by withdrawing bad symbols and stopwords using NLTK.

In [0]:
#Converting to string form
topics_data['title'] = topics_data['title'].apply(string_form)
topics_data['body'] = topics_data['body'].apply(string_form)
topics_data['comments'] = topics_data['comments'].apply(string_form)
#Clean the data
topics_data['title'] = topics_data['title'].apply(clean_text)
topics_data['body'] = topics_data['body'].apply(clean_text)
topics_data['comments'] = topics_data['comments'].apply(clean_text)
#Combining the title, comments, and url feature into one single attribute
feature_combine = topics_data["title"] + topics_data["comments"] + topics_data["url"]
topics_data = topics_data.assign(feature_combine = feature_combine)

###Machine Learning Models
Using scikit-learn libraries

### 1. Naive Bayes Classifier

In [0]:
def nb_classifier(X_train, X_test, y_train, y_test):
  
  from sklearn.naive_bayes import MultinomialNB


  nb = Pipeline([('vect', CountVectorizer()),
                 ('tfidf', TfidfTransformer()),
                 ('clf', MultinomialNB()),
                ])
  nb.fit(X_train, y_train)

  y_pred = nb.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

### 2. Linear SVM

In [0]:
def linear_svm(X_train, X_test, y_train, y_test):
  
  from sklearn.linear_model import SGDClassifier

  sgd = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
                 ])
  sgd.fit(X_train, y_train)

  y_pred = sgd.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

### 3. Logistic Regression

In [0]:
def logisticreg(X_train, X_test, y_train, y_test):

  from sklearn.linear_model import LogisticRegression

  logreg = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', LogisticRegression(n_jobs=1, C=1e5)),
                 ])
  logreg.fit(X_train, y_train)

  y_pred = logreg.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

### 4. Random Forest

In [0]:
def randomforest(X_train, X_test, y_train, y_test):
  
  from sklearn.ensemble import RandomForestClassifier
  
  ranfor = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', RandomForestClassifier(n_estimators = 1000, random_state = 42)),
                 ])
  ranfor.fit(X_train, y_train)

  y_pred = ranfor.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

### 5. MLP Classifier

In [0]:
def mlpclassifier(X_train, X_test, y_train, y_test):
  
  from sklearn.neural_network import MLPClassifier
  
  mlp = Pipeline([('vect', CountVectorizer()),
                  ('tfidf', TfidfTransformer()),
                  ('clf', MLPClassifier(hidden_layer_sizes=(30,30,30))),
                 ])
  mlp.fit(X_train, y_train)

  y_pred = mlp.predict(X_test)

  print('accuracy %s' % accuracy_score(y_pred, y_test))
  print(classification_report(y_test, y_pred,target_names=flairs))

### Training and testing the five ML Models

In [0]:
def train_test(X,y):
  #Splitting the training and testing data in ratio 7:3 
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state = 42)
  #Printing the results of all the algorithms based on a feature
  print("Naive Bayes Classifier")
  nb_classifier(X_train, X_test, y_train, y_test)
  print("Linear SVM")
  linear_svm(X_train, X_test, y_train, y_test)
  print("Logistic Regression")
  logisticreg(X_train, X_test, y_train, y_test)
  print("Random Forest")
  randomforest(X_train, X_test, y_train, y_test)
  print("Multi-Layer Perceptron Classifier")
  mlpclassifier(X_train, X_test, y_train, y_test)

### Results

In [34]:
#Given flairs list
cat = topics_data.flair
#Given features
V = topics_data.feature_combine
W = topics_data.comments
X = topics_data.title
Y = topics_data.body
Z = topics_data.url
print("Detection using Title as Feature")
train_test(X,cat)
print("Detection using Body as Feature")
train_test(Y,cat)
print("Detection using URL as Feature")
train_test(Z,cat)
print("Detection using Comments as Feature")
train_test(V,cat)

Detection using Title as Feature
Naive Bayes Classifier
accuracy 0.6126760563380281
                    precision    recall  f1-score   support

          AskIndia       0.92      0.63      0.75        19
     Non-Political       0.33      0.25      0.29        12
     [R]eddiquette       0.50      0.36      0.42        11
         Scheduled       0.32      1.00      0.48         6
       Photography       0.92      0.60      0.73        20
Science/Technology       0.79      1.00      0.88        11
          Politics       0.50      0.83      0.62        12
  Business/Finance       0.31      0.56      0.40         9
    Policy/Economy       0.88      0.54      0.67        13
            Sports       0.80      0.57      0.67        14
              Food       0.75      0.82      0.78        11
               AMA       0.00      0.00      0.00         4

          accuracy                           0.61       142
         macro avg       0.58      0.60      0.56       142
      weighted

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0

accuracy 0.7112676056338029
                    precision    recall  f1-score   support

          AskIndia       0.88      0.74      0.80        19
     Non-Political       0.40      0.50      0.44        12
     [R]eddiquette       0.31      0.36      0.33        11
         Scheduled       0.83      0.83      0.83         6
       Photography       0.95      1.00      0.98        20
Science/Technology       1.00      1.00      1.00        11
          Politics       0.47      0.75      0.58        12
  Business/Finance       0.58      0.78      0.67         9
    Policy/Economy       0.88      0.54      0.67        13
            Sports       0.80      0.57      0.67        14
              Food       0.91      0.91      0.91        11
               AMA       0.00      0.00      0.00         4

          accuracy                           0.71       142
         macro avg       0.67      0.67      0.66       142
      weighted avg       0.73      0.71      0.71       142

Random Fo

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy 0.5070422535211268
                    precision    recall  f1-score   support

          AskIndia       0.85      0.58      0.69        19
     Non-Political       0.17      0.17      0.17        12
     [R]eddiquette       0.33      0.18      0.24        11
         Scheduled       0.56      0.83      0.67         6
       Photography       0.44      0.40      0.42        20
Science/Technology       1.00      0.73      0.84        11
          Politics       0.36      0.67      0.47        12
  Business/Finance       0.40      0.67      0.50         9
    Policy/Economy       0.55      0.46      0.50        13
            Sports       0.47      0.57      0.52        14
              Food       0.78      0.64      0.70        11
               AMA       0.50      0.25      0.33         4

          accuracy                           0.51       142
         macro avg       0.53      0.51      0.50       142
      weighted avg       0.54      0.51      0.51       142

Detection

...

accuracy 0.21830985915492956
                    precision    recall  f1-score   support

          AskIndia       0.50      0.05      0.10        19
     Non-Political       0.14      1.00      0.25        12
     [R]eddiquette       0.00      0.00      0.00        11
         Scheduled       0.00      0.00      0.00         6
       Photography       0.44      0.20      0.28        20
Science/Technology       1.00      0.09      0.17        11
          Politics       0.43      0.25      0.32        12
  Business/Finance       0.20      0.11      0.14         9
    Policy/Economy       0.25      0.15      0.19        13
            Sports       1.00      0.29      0.44        14
              Food       0.60      0.27      0.37        11
               AMA       0.00      0.00      0.00         4

          accuracy                           0.22       142
         macro avg       0.38      0.20      0.19       142
      weighted avg       0.44      0.22      0.21       142

Random F

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


accuracy 0.1619718309859155
                    precision    recall  f1-score   support

          AskIndia       0.50      0.05      0.10        19
     Non-Political       0.00      0.00      0.00        12
     [R]eddiquette       0.00      0.00      0.00        11
         Scheduled       0.00      0.00      0.00         6
       Photography       0.33      0.35      0.34        20
Science/Technology       1.00      0.09      0.17        11
          Politics       0.09      0.67      0.15        12
  Business/Finance       0.14      0.11      0.12         9
    Policy/Economy       0.29      0.15      0.20        13
            Sports       1.00      0.14      0.25        14
              Food       0.50      0.09      0.15        11
               AMA       0.00      0.00      0.00         4

          accuracy                           0.16       142
         macro avg       0.32      0.14      0.12       142
      weighted avg       0.37      0.16      0.15       142

Detection

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


accuracy 0.6619718309859155
                    precision    recall  f1-score   support

          AskIndia       0.80      0.63      0.71        19
     Non-Political       0.47      0.67      0.55        12
     [R]eddiquette       0.60      0.27      0.37        11
         Scheduled       0.62      0.83      0.71         6
       Photography       0.83      0.50      0.62        20
Science/Technology       0.85      1.00      0.92        11
          Politics       0.64      0.75      0.69        12
  Business/Finance       0.44      0.78      0.56         9
    Policy/Economy       0.67      0.46      0.55        13
            Sports       0.59      0.71      0.65        14
              Food       0.79      1.00      0.88        11
               AMA       1.00      0.50      0.67         4

          accuracy                           0.66       142
         macro avg       0.69      0.68      0.66       142
      weighted avg       0.69      0.66      0.65       142

Random Fo

In [35]:
pwd

'/content'

Clone the github repository to get the saved model.

In [36]:
!git clone https://github.com/AshuKV/Reddit-Flair-Predictor.git

Cloning into 'Reddit-Flair-Predictor'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 39 (delta 10), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (39/39), done.


### For detecting Reddit India Post Flair

In [0]:
#Function to detect the flair
def detect_flair(id):
  #Store the submission ID
  submission = reddit.submission(id=id)
  #Dictionary for storing all features
  data = {}
  data['title'] = submission.title
  data['url'] = submission.url
  #Getting only top level comments
  submission.comments.replace_more(limit=None)
  comment = ''
  for top_level_comment in submission.comments:
    comment = comment + ' ' + top_level_comment.body
  data["comment"] = comment
  data['title'] = clean_text(data['title'])
  data['comment'] = clean_text(data['comment'])
  data['combine'] = data['title'] + data['comment'] + data['url']
  #return the detected flair
  return loaded_model.predict([data['combine']])

###Load the saved Model
Import the pickle library for loading the saved model.

In [39]:
import pickle
loaded_model = pickle.load(open('/content/Reddit-Flair-Predictor/Finalised_model/model_LOGREG.sav', 'rb'))

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.logistic module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:31

###Final Detection
Detect top 5 Reddit flair

In [41]:
subreddit = reddit.subreddit('india')

for submission in subreddit.top(limit=5):
    print(detect_flair(submission.id), submission.permalink)

['AMA'] /r/india/comments/981o7s/will_donate_thrice_the_number_of_upvotes_amount/
['AMA'] /r/india/comments/6f10op/indian_reply_to_nytimes_cartoon_on_paris_climate/
['AskIndia'] /r/india/comments/8pymkp/the_essence_of_the_indian_soap_opera_distilled/
['[R]eddiquette'] /r/india/comments/f9outu/fuck_all_religion/
['Politics'] /r/india/comments/eev8g5/german_exchange_student_at_iit_madras_is_being/
